In [1]:
from mem0 import Memory
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank
import os
from dotenv import load_dotenv

load_dotenv()



config = {
    "llm": {
        "provider": "nvidia",
            "config": {
                "model": "meta/llama-3.1-405b-instruct",
                "temperature": 0.1,
                "max_tokens": 2080,
            }
    },
    "embedder" : {
        "provider": "nvidia",
        },
    "vector_store": {
    "provider": "chroma",
    "config": {
        "collection_name": "test",
        "path": "db",
    }
    }
}


memory = Memory.from_config(config)


nvidia/nv-embed-v1


In [2]:

# For a user
result = memory.add("Likes to play cricket on weekends", user_id="alice", metadata={"category": "hobbies"})

messages = [
    {"role": "user", "content": "Hi, I'm Alex. I like to play piano"},
    {"role": "assistant", "content": "Hello Alex! It's great to know that you enjoy playing cricket on weekends. I'll remember that for future reference."}
 ]
result=memory.add(messages, user_id="alice")
print(result)


C:\Users\zcharpy\AppData\Local\Temp\ipykernel_28348\2898316666.py:2: DeprecationWarning: The current add API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  result = memory.add("Likes to play cricket on weekends", user_id="alice", metadata={"category": "hobbies"})


[{'id': 'cdb9f4f4-c46b-44fa-8dfb-2ba7979386fe', 'memory': 'Likes to play piano', 'event': 'DELETE'}, {'id': '873915df-0422-4a8d-88ec-45438640a730', 'memory': 'Likes to play cricket on weekends', 'event': 'DELETE'}, {'id': '57ad3e8a-17a5-469b-9817-c41dfeebd62c', 'memory': 'Name is Alex', 'event': 'DELETE'}]


C:\Users\zcharpy\AppData\Local\Temp\ipykernel_28348\2898316666.py:8: DeprecationWarning: The current add API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  result=memory.add(messages, user_id="alice")


### default  categories 

- personal_details
- family
- professional_details
- sports
- travel
- food
- music
- health
- technology
- hobbies
- fashion
- entertainment
- milestones
- user_preferences
- misc


In [3]:
result = memory.add("My name is Morty", user_id="rik", metadata={"category": "personal_dtails"})
result = memory.add("I have 1 daughter and 2 grandchildren", user_id="rik", metadata={"category": "family"})
result = memory.add("I am retired scientist", user_id="rik", metadata={"category": "professional_details"})
result = memory.add("My favorite team is FC Barcelona", user_id="rik", metadata={"category": "sports"})
result = memory.add("I like rocky road ice cream", user_id="rik", metadata={"category": "food"})
result = memory.add("My enjoy listen to rock and roll music", user_id="rik", metadata={"category": "music"})
result = memory.add("I don't have any fashion sense", user_id="rik", metadata={"category": "fashion"})
result = memory.add("I like to do interstellar travel in my space car", user_id="rik", metadata={"category": "entertainment"})


C:\Users\zcharpy\AppData\Local\Temp\ipykernel_28348\4293368332.py:1: DeprecationWarning: The current add API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  result = memory.add("My name is Morty", user_id="rik", metadata={"category": "personal_dtails"})
C:\Users\zcharpy\AppData\Local\Temp\ipykernel_28348\4293368332.py:2: DeprecationWarning: The current add API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  result = memory.add("I have 1 daughter and 2 grandchildren", user_id="rik", metadata={"category": "family"})
C:\Users\zcharpy\AppData\Local\Temp\ipykernel_28348\4293368332.py:3: DeprecationWarning: The current add API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  resul

In [4]:
query="tell me about Morty's family"
mem_search_output=memory.search(
    query=query,
    user_id="rik",
    limit=2  # Adjust based on needs
)
mem_search_output

C:\Users\zcharpy\AppData\Local\Temp\ipykernel_28348\641792229.py:2: DeprecationWarning: The current get_all API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  mem_search_output=memory.search(


[{'id': 'a5874704-b982-4547-813d-105ee63d2b89',
  'memory': 'Name is Morty',
  'hash': 'de87fa73e3d2a2c1ddf271ed643b4945',
  'metadata': {'category': 'personal_dtails'},
  'score': 0.9302722576429723,
  'created_at': '2024-11-26T04:15:24.981154-08:00',
  'updated_at': None,
  'user_id': 'rik'},
 {'id': 'cc54e09f-259d-451e-ac4f-62611ee139af',
  'memory': 'Has 2 grandchildren',
  'hash': 'e6a34e89cbdb06c90742400c1c7798c9',
  'metadata': {'category': 'family'},
  'score': 1.7430827373716298,
  'created_at': '2024-11-26T04:15:33.481988-08:00',
  'updated_at': None,
  'user_id': 'rik'}]

## wrap into chatbot with memory 

In [13]:
import os
from typing import List, Dict
from mem0 import Memory
from datetime import datetime
class SupportChatbot:
    def __init__(self):
        # Initialize Mem0 with Anthropic's Claude
        self.config = config = {
            "llm": {
                "provider": "nvidia",
                    "config": {
                        "model": "meta/llama-3.1-405b-instruct",
                        "temperature": 0.1,
                        "max_tokens": 2080,
                    }
            },
            "embedder" : {
                "provider": "nvidia",
                },
            "vector_store": {
            "provider": "chroma",
            "config": {
                "collection_name": "test",
                "path": "db",
            }
            }
        }
        self.memory = Memory.from_config(self.config)
        self.client = memory.llm

        # Define support context
        self.system_context = """
        You are a helpful customer support agent. Use the following guidelines:
        - Be polite and professional
        - Show empathy for customer issues
        - Reference past interactions when relevant
        - Maintain consistent information across conversations
        - If you're unsure about something, ask for clarification
        - Keep track of open issues and follow-ups
        """

    def store_customer_interaction(self,
                                 user_id: str,
                                 message: str,
                                 response: str,
                                 metadata: Dict = None):
        """Store customer interaction in memory."""
        if metadata is None:
            metadata = {}

        # Add timestamp to metadata
        metadata["timestamp"] = datetime.now().isoformat()

        # Format conversation for storage
        conversation = [
            {"role": "user", "content": message},
            {"role": "assistant", "content": response}
        ]

        # Store in Mem0
        self.memory.add(
            conversation,
            user_id=user_id,
            metadata=metadata
        )

    def get_relevant_history(self, user_id: str, query: str) -> List[Dict]:
        """Retrieve relevant past interactions."""
        return self.memory.search(
            query=query,
            user_id=user_id,
            limit=5  # Adjust based on needs
        )

    def handle_customer_query(self, user_id: str, query: str) -> str:
        """Process customer query with context from past interactions."""

        # Get relevant past interactions
        relevant_history = self.get_relevant_history(user_id, query)

        # Build context from relevant history
        context = "Previous relevant interactions:\n"
        for memory in relevant_history:
            context += f"Customer: {memory['memory']}\n"
            context += f"Support: {memory['memory']}\n"
            context += "---\n"

        # Prepare prompt with context and current query
        prompt = f"""
        {self.system_context}

        {context}

        Current customer query: {query}

        Provide a helpful response that takes into account any relevant past interactions.
        """

        # Generate response using Claude
        response = self.client.generate_response(
            messages=[{"role": "user", "content": prompt}]
        )
        print("##### response \n", response)
        # Store interaction
        self.store_customer_interaction(
            user_id=user_id,
            message=query,
            response=response,
            metadata={"type": "support_query"}
        )
        
        return response

In [15]:
chatbot = SupportChatbot()
user_id = "customer_bot"
print("Welcome to Customer Support! Type 'exit' to end the conversation.")

while True:
    # Get user input
    query = input()
    print("Customer:", query)
    
    # Check if user wants to exit
    if query.lower() == 'exit':
        print("Thank you for using our support service. Goodbye!")
        break
    
    # Handle the query and print the response
    response = chatbot.handle_customer_query(user_id, query)
    print("Support:", response, "\n\n")
    

nvidia/nv-embed-v1
Welcome to Customer Support! Type 'exit' to end the conversation.


 Hi, my name is Zenodia and I'm having trouble connecting my new smartwatch to the mobile app. It keeps showing a connection error.


Customer: Hi, my name is Zenodia and I'm having trouble connecting my new smartwatch to the mobile app. It keeps showing a connection error.


C:\Users\zcharpy\AppData\Local\Temp\ipykernel_28348\80001386.py:69: DeprecationWarning: The current get_all API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  return self.memory.search(


##### response 
 Hi Zenodia, thank you for reaching out to us again. I'm sorry to hear that you're experiencing issues with connecting your new smartwatch to the mobile app. I recall that you had a similar issue in the past, but we were able to resolve it successfully.

Before we proceed, I just want to confirm that you've already tried the initial troubleshooting steps we discussed previously, is that correct? If so, I'd like to dive a bit deeper to identify the root cause of the issue this time around.

Can you please tell me more about the error message you're seeing on the app? Is it the same error message as before, or is it a different one? Additionally, have you made any changes to your smartwatch or mobile device settings recently that might be contributing to the issue?


...

##### response 
 Thank you for reaching out to us again. I'm glad to hear that you've already gone through the troubleshooting steps we previously discussed. I've reviewed our past conversations, and I see that you had issues with connecting your smartwatch to the mobile app, which we were able to resolve. Additionally, you mentioned that your smartwatch battery wasn't charging to 100%.

Regarding your current question, I'd like to clarify a few things before we consider replacing the battery. Can you please tell me more about the current battery performance? Are you still experiencing issues with it not charging to 100%, or has the problem persisted even after resolving the connection issue with the mobile app?

Also, have you noticed any other symptoms, such as rapid battery drain or unusual power consumption patterns? This information will help me better understand the situation and provide a more informed recommendation on whether replacing the battery is the best course of action.

C:\Users\zcharpy\AppData\Local\Temp\ipykernel_28348\80001386.py:61: DeprecationWarning: The current add API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  self.memory.add(


Support: Thank you for reaching out to us again. I'm glad to hear that you've already gone through the troubleshooting steps we previously discussed. I've reviewed our past conversations, and I see that you had issues with connecting your smartwatch to the mobile app, which we were able to resolve. Additionally, you mentioned that your smartwatch battery wasn't charging to 100%.

Regarding your current question, I'd like to clarify a few things before we consider replacing the battery. Can you please tell me more about the current battery performance? Are you still experiencing issues with it not charging to 100%, or has the problem persisted even after resolving the connection issue with the mobile app?

Also, have you noticed any other symptoms, such as rapid battery drain or unusual power consumption patterns? This information will help me better understand the situation and provide a more informed recommendation on whether replacing the battery is the best course of action. 




 exit


Customer: exit
Thank you for using our support service. Goodbye!


In [20]:
chatbot.get_relevant_history(user_id = "customer_bot",query="what trouble did user encountered")

C:\Users\zcharpy\AppData\Local\Temp\ipykernel_28348\80001386.py:69: DeprecationWarning: The current get_all API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  return self.memory.search(


[{'id': 'b2c41b16-cbe7-44f5-b9a2-c9ea0a2e34d5',
  'memory': 'Tried initial troubleshooting steps previously',
  'hash': 'c59fa3da346f80d7eabd24b04b8ef056',
  'metadata': {'timestamp': '2024-11-26T14:40:20.191234',
   'type': 'support_query'},
  'score': 1.4778110179304884,
  'created_at': '2024-11-26T05:40:39.074220-08:00',
  'updated_at': '2024-11-26T05:40:39.064215-08:00',
  'user_id': 'customer_bot'},
 {'id': 'a9d00be7-4497-4033-98f8-06bc2dc7f9fd',
  'memory': 'Had previously discussed initial troubleshooting steps',
  'hash': '3547d1b0f8602ef0731129c5947af09b',
  'metadata': {'timestamp': '2024-11-26T14:42:49.088993',
   'type': 'support_query'},
  'score': 1.4818143869148983,
  'created_at': '2024-11-26T05:43:54.474515-08:00',
  'updated_at': '2024-11-26T05:43:54.456077-08:00',
  'user_id': 'customer_bot'},
 {'id': 'e5e27087-20ae-4a86-b682-42c8cce108fb',
  'memory': 'Having trouble connecting new smartwatch to mobile app',
  'hash': '48f8a6f913bc2ba64ea42b84327a2efa',
  'metadata'